In [ ]:
import digitalhub as dh

project = dh.get_or_create_project("llm")

## Text Classification LLM

In [ ]:
llm_function = project.new_function("llm-classification",
                                    kind="huggingfaceserve",
                                    model_name="mymodel",
                                    path="huggingface://distilbert/distilbert-base-uncased-finetuned-sst-2-english")

In [ ]:
llm_run = llm_function.run(action="serve", profile="1xa100", wait=True)

In [ ]:
json = {
    "inputs": [
        {
            "name": "input-0",
            "shape": [2],
            "datatype": "BYTES",
            "data": ["Hello, my dog is cute", "I am feeling sad"],
        },
    ]
}

llm_run.invoke(model_name="mymodel", json=json).json()

## Text Generation LLM

In [ ]:
llm_function = project.new_function("llm-generation",
                                    kind="huggingfaceserve",
                                    model_name="mymodel",
                                    path="huggingface://meta-llama/meta-llama-3-8b-instruct")

In [ ]:
hf_token = "<HUGGINGFACE TOKEN>"
llm_run = llm_function.run(action="serve",
                           profile="1xa100-80GB",
                           envs = [{"name": "HF_TOKEN", "value": hf_token}],
                           wait=True)

In [ ]:
service_url = llm_run.refresh().status.to_dict()["service"]["url"]
url = f"http://{service_url}/openai/v1/completions"

json = {
    "model": "mymodel",
    "prompt": "Hello! How are you?",
    "stream": False,
    "max_tokens": 30
}

llm_run.invoke(url=url, json=json).json()

In [ ]:
service_url = llm_run.refresh().status.to_dict()["service"]["url"]
url = f'http://{service_url}/openai/v1/chat/completions'

model_name = "mymodel"

json = {
    "model": "mymodel",
    "messages": [
        {"role": "system", "content": "You are an assistant that speaks like Shakespeare."},
        {"role": "user", "content": "Write a poem about colors"}
    ],
    "max_tokens": 30,
    "stream": False
}

llm_run.invoke(url=url, json=json).json()

## Fine-tuned LLM model

In [ ]:
from pathlib import Path
Path("src").mkdir(exist_ok=True)

In [ ]:
%%writefile "src/train_model.py"

import os

import evaluate
import numpy as np
from datasets import load_dataset
from digitalhub_runtime_python import handler
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments


@handler()
def train(project):
    tokenizer = AutoTokenizer.from_pretrained("google-bert/bert-base-cased")
    metric = evaluate.load("accuracy")

    def tokenize_function(examples):
        return tokenizer(examples["text"], padding="max_length", truncation=True)

    def compute_metrics(eval_pred):
        logits, labels = eval_pred
        predictions = np.argmax(logits, axis=-1)
        return metric.compute(predictions=predictions, references=labels)

    dataset = load_dataset("yelp_review_full")
    tokenized_datasets = dataset.map(tokenize_function, batched=True)

    model = AutoModelForSequenceClassification.from_pretrained("google-bert/bert-base-cased", num_labels=5)

    training_args = TrainingArguments(output_dir="test_trainer")

    small_train_dataset = tokenized_datasets["train"].shuffle(seed=42).select(range(1000))
    small_eval_dataset = tokenized_datasets["test"].shuffle(seed=42).select(range(1000))

    training_args = TrainingArguments(output_dir="test_trainer", eval_strategy="epoch")

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=small_train_dataset,
        eval_dataset=small_eval_dataset,
        compute_metrics=compute_metrics,
    )

    trainer.train()

    save_model = "model"
    if not os.path.exists(save_model):
        os.makedirs(save_model)

    save_dir = "model"
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    trainer.save_model(save_dir)
    tokenizer.save_pretrained(save_dir)

    project.log_model(
        name="test_llm_model",
        kind="huggingface",
        base_model="google-bert/bert-base-cased",
        source=save_dir,
    )

In [ ]:
train_func = project.new_function(name="train_model",
                                  kind="python",
                                  python_version="PYTHON3_10",
                                  code_src="src/train_model.py",
                                  handler="train",
                                  requirements=["evaluate", "transformers[torch]", "torch", "torchvision", "accelerate"])

In [ ]:
train_run = train_func.run(action="job", profile="1xa100", wait=True)

In [ ]:
llm_model_path = project.get_model("test_llm_model").spec.path

In [ ]:
llm_function = project.new_function("llm-classification",
                                    kind="huggingfaceserve",
                                    model_name="mymodel",
                                    path=llm_model_path)

In [ ]:
llm_run = llm_function.run(action="serve", profile="1xa100", volumes=[{
                        "volume_type": "persistent_volume_claim",
                        "name": "volume-llmpa",
                        "mount_path": "/shared",
                        "spec": { "size": "10Gi" }}]
                    )

In [ ]:
json = {
    "inputs": [
        {
            "name": "input-0",
            "shape": [2],
            "datatype": "BYTES",
            "data": ["Hello, my dog is cute", "I am feeling sad"],
        }
    ]
}

llm_run.refresh().invoke(model_name="mymodel", json=json).json()